# 작업개요

- TourAPI의 관광지 데이터가 전체 유형(관광지, 식당, 쇼핑, 레포츠, 여행 코스 등) 포함 약 5.3만개로 POI 대상지가 매우 감소
- 이에 따라 관광로그 데이터를 전국구로 수집할 필요성 느낌
- AI Hub의 관광로그 데이터 4종(수도권, 동부, 서부, 제주/도서지역) 을 병합해 하나의 데이터셋으로 생성 필요
- 생성된 데이터셋에 대한 간단한 EDA 진행

### 작업 목표
- AI Hub 관광로그 데이터 4종 병합
- 병합된 데이터셋에 대한 간단한 EDA
- 불필요 데이터 전처리
- **추천모델 학습용 데이터셋 확보**

In [ ]:
import pandas as pd
import numpy as np
import random

# 출력할 최대 열 수 설정
pd.set_option('display.max_columns', None)

# 출력할 최대 행 수 설정
pd.set_option('display.max_rows', None)

import warnings

# 모든 경고 무시
warnings.filterwarnings("ignore")

# 데이터 전처리

### 카테고리 데이터 정보 테이블

- 일부 데이터에 입력된 카테고리컬 데이터의 의미가 담긴 테이블

In [ ]:
TC_CODEA = pd.read_csv(
    '../data/travel_log/277.국내 여행로그 데이터(수도권)/Training/tc_codea_코드A.csv', encoding='utf-8')
TC_CODEB = pd.read_csv(
    '../data/travel_log/277.국내 여행로그 데이터(수도권)/Training/tc_codeb_코드B.csv', encoding='utf-8')

# 코드_a의 cd_nm을 'code_nm_a'로 이름을 변경하여 복사
code_a_copy = TC_CODEA.rename(
    columns={'cd_nm': 'code_nm_a', 'cd_memo': 'cd_memo_a'}).copy()

# 코드_b의 cd_nm을 'code_nm_b'로 이름을 변경하여 복사
code_b_copy = TC_CODEB.rename(
    columns={'cd_nm': 'code_nm_b', 'cd_memo': 'cd_memo_b'}).copy()

# 코드_a와 코드_b 데이터프레임을 cd_a를 기준으로 병합
merged_df = pd.merge(code_b_copy, code_a_copy, on='cd_a', how='left')

# 코드 값에 따라 cd_nm_a, cd_nm_b, cd_a, cd_b를 그룹바이하여 출력하는 작업
basic_code = merged_df.groupby('cd_a').apply(
    lambda x: x[['cd_b', 'code_nm_a', 'code_nm_b']].reset_index(drop=True))
# 결과 출력
basic_code.head()

## 데이터 불러오기

In [ ]:
import pandas as pd
import os

# 기본 경로 설정
basic_path = '../data/travel_log'
regions = ['277.국내 여행로그 데이터(수도권)', '278.국내 여행로그 데이터(동부권)', '279.국내 여행로그 데이터(서부권)', '280.국내 여행로그 데이터(제주도 및 도서지역)']
suffixes = ['A', 'B', 'C', 'D']

# 파일명 패턴
file_patterns = ['tn_travel_여행', 'tn_traveller_master_여행객 Master', 'tn_visit_area_info_방문지정보']

# 각 DataFrame을 저장할 딕셔너리 생성
dfs = {pattern: [] for pattern in file_patterns}

# 지역별, 파일 패턴별로 데이터 로드 및 병합
for region, suffix in zip(regions, suffixes):
    for pattern in file_patterns:
        files = [
            os.path.join(basic_path, region, 'Training', f'{pattern}_{suffix}.csv'),
            os.path.join(basic_path, region, 'Validation', f'{pattern}_{suffix}.csv')
        ]
        # 파일 병합
        df_concat = pd.concat([pd.read_csv(file) for file in files], axis=0)
        dfs[pattern].append(df_concat)

# 최종적으로 각 패턴별로 데이터 병합하여 3개의 DataFrame 생성
travel = pd.concat(dfs['tn_travel_여행'], axis=0)
travel_master = pd.concat(dfs['tn_traveller_master_여행객 Master'], axis=0)
visit_area = pd.concat(dfs['tn_visit_area_info_방문지정보'], axis=0)


### 여행정보

In [ ]:
display(travel.describe())
display(travel.info())
travel.head()

### 여행객 정보

In [ ]:
display(travel_master.describe())
display(travel_master.info())
travel_master.head()

![](https://i.imgur.com/2dEDL24.png)

In [ ]:
# TRAVEL_STYL_1부터 TRAVEL_STYL_8까지의 컬럼 이름 리스트 생성
travel_style_columns = [f'TRAVEL_STYL_{i}' for i in range(1, 9)]

# 각 컬럼별 unique 값 출력
for column in travel_style_columns:
    unique_values = sorted(travel_master[column].unique())
    print(f'{column} unique values: {unique_values}')

- 각 스타일별로 1에 가까울수록 해당 스타일을 선호하지 않고, 7에 가까울수록 해당 스타일을 선호하는 것

### 방문지 정보

In [ ]:
display(visit_area.describe())
display(visit_area.info())
visit_area.head()

### 여행지가 아닌 데이터 필터링

- `visit_area` 테이블에서 `VISIT_AREA_NM` 을 보면 일부 여행기록은 `집`, `병점역 1호선` 등 관광지가 아닌 데이터들이 다수 포함되어있음.
- 여행의 동선을 기록한 데이터라 출발지, 중간 휴식지, 경유지 등이 포함되어 있는 것
- 따라서 여행지가 아닌데 이터들을 필터링할 필요가 있음
- 카테고리 데이터 정보테이블을 기반으로 관광지가 아닌 데이터들을 필터링할 수 있음
- `VISIT_AREA_TYPE_CD` 은 방문지 유형에 대한 코드로 카테고리 데이터 정보테이블을 통해 각 코드가 어떤 유형인지 파악할 수 있음

In [ ]:
# 여행지가 아닌 데이터 필터링 위한 기준표 출력
TRAVEL_VISIT_group = basic_code.loc['VIS']
TRAVEL_VISIT_group

- 1번부터 8번, 13번은 관광지로 포함시킬 수 있으나 그 외 지역은 관광지로 보기 어려움
- 따라서 visit_area 에서 1~8, 13을 제외한 나머지 코드는 drop하여 필터링을 진행

In [ ]:
# VISIT_AREA_TYPE_CD 열의 값이 1, 2, 3, 4, 5, 6, 7, 8, 13인 행만 추출
valid_values = [1, 2, 3, 4, 5, 6, 7, 8, 13]
visit_area_filtered = visit_area[visit_area['VISIT_AREA_TYPE_CD'].isin(valid_values)]
visit_area_filtered.info()

print(f' \n 필터링된 VISIT_AREA_TYPE_CD의 고유 코드 리스트는 : {sorted(visit_area_filtered["VISIT_AREA_TYPE_CD"].unique())}')

## 데이터 병합

In [ ]:
df = pd.merge(visit_area_filtered,  travel, on='TRAVEL_ID', how='left')
travel_log = pd.merge(df, travel_master, on='TRAVELER_ID', how='left')

display(travel_log.describe())
display(travel_log.info())
travel_log.head()

In [ ]:
travel_log.columns

In [ ]:
travel_log.rename(columns={'DGSTFN': 'STARS'}, inplace=True)
travel_log.rename(columns={'VISIT_START_YMD': 'TRAVEL_YMD'}, inplace=True)

In [ ]:
db_travel = travel_log[['TRAVEL_ID', 'TRAVEL_NM', 'TRAVELER_ID', 'VISIT_AREA_ID', 'VISIT_AREA_NM', 'VISIT_AREA_TYPE_CD',
                                            'TRAVEL_YMD','POI_ID', 'POI_NM', 'X_COORD', 'Y_COORD',
                                            'SGG_CD', 'ROAD_NM_CD','ROAD_NM_ADDR', 'LOTNO_CD', 'LOTNO_ADDR', 
                                            'REVISIT_INTENTION', 'RCMDTN_INTENTION', 'TRAVEL_MISSION_CHECK',
                                            'GENDER', 'AGE_GRP', 
                                            'TRAVEL_STYL_1', 'TRAVEL_STYL_2', 'TRAVEL_STYL_3', 'TRAVEL_STYL_4', 'TRAVEL_STYL_5',
                                            'TRAVEL_STYL_6', 'TRAVEL_STYL_7', 'TRAVEL_STYL_8', 
                                            'TRAVEL_MOTIVE_1', 'TRAVEL_MOTIVE_2', 'TRAVEL_MOTIVE_3'                        
                                            ,'STARS'
                                           ]].copy()

db_travel.info()

In [ ]:
#db_travel['user_id'] = db_travel['TRAVELER_ID']

db_travel.head()

In [ ]:
# 구축 결과 테스트
db_travel[db_travel['VISIT_AREA_NM'].str.contains("카멜리아")].head()

In [ ]:
db_travel.info()

In [ ]:
db_travel.to_csv('travel_log.csv', encoding='utf-8-sig', index=False)

### 주요 변수에 대한 해석(feat. AI Hub문서)

> **✔️ Bold 처리된 컬럼은 필수 데이터**
> 사용 불필요 데이터는 제외하고 해석

- VISIT_AREA_ID : 여행경로상의 모든 방문지에 대한 고유ID, POI ID라고 볼 수 있음
- TRAVEL_ID : 여행로그에 대한 고유ID, 하나의 ID에 여러 VISIT_AREA_ID 에 대한 정보가 포함되어 있음
- VISIT_AREA_ID : 각 여행의 POI방문 순서 (필요X)
- **VISIT_AREA_NM : VISIT_AREA_ID의 장소명**
    - `숙소` 등 관광지라기보단 여행지를 관광객이 기록한 명칭이 입력되어 있음
    - 관광지명이 아닌 데이터에 대한 전처리 필요해 보임
- ROAD_NM_ADDR : 방문장소의 도로명 주소
    - TourAPI 데이터와 장소가 같더라도 상세주소 및 좌표정보가 다름(관광객이 직접 수집한정보라 오차가 발생한듯 보임)
    - 차후 TourAPI의 POI와 병합할때 전처리 방법 고민 필요
- LOTNO_ADDR : 방문장소의 지번주소
- X_COORD : 경도
- Y_COORD : 위도
- ROAD_NM_CD : 도로명 코드
- LOTNO_CD : 지번 코드
- POI_ID : 방문장소 고유ID
- POI_NM : 방문장소의 공식 명칭
- **VISIT_AREA_TYPE_CD : 방문지유형코드 (코드A & 코드B로 확인 가능)**
- REVISIT_YN : 재방문여부
- VISIT_CHC_REASON_CD : 방문선택이유 코드 (코드A & 코드B로 확인 가능)
- **DGSTFN : 만족도 (y label이 될 값)**
- REVISIT_INTENTION : 재방문 의향
- RCMDTN_INTENTION : 추천 의향
- SGG_CD : 시군구 코드
- TRAVEL_NM : 여행명 (TRAVELER_ID 와 결합되어 TRAVEL_ID가 됨)
- TRAVELER_ID : 여행객 고유 ID
- TRAVEL_PURPOSE : 여행목적 (총 21개의 카테고리, 코드A & 코드B로 확인 가능)
- TRAVEL_PERSONA : 여행 행태별 페르소나 👉 여행객의 여행 스타일 유형
- TRAVEL_MISSION : 여행미션 (여행목적으로 21개의 카테고리, 코드A & 코드B로 확인 가능)
- TRAVEL_MISSION_CHECK : 미션우선도 Check(전체 여행기록중 설정한 미션대로 여행했는지 여부로 TRAVEL_MISSION과 동일 카테고리사용)
    - 즉, `TRAVEL_MISSION` 는 자료 수집시 여행객에게 부여된 미션이고, `TRAVEL_MISSION_CHECK` 이 실제로 여행하면서 여행객이 수행한 미션에 대한 정보
- RESIDENCE_SGG_CD : 여행객 거주지 시군구코드
- GENDER : 여행객 성별
- AGE_GRP : 여행객 연령대 (20~60대 , nan)
- TRAVEL_TERM : 여형빈도_기간 (4개 카테고리 : 1주일, 한달, 1년, 기타) + 코드A & 코드B로 확인 가능
- TRAVEL_NUM : 여행빈도 (1~20 순차적으로 있고, 나머지는 24, 30, 40, 100 등 범주가 제각각)
- TRAVELER_ID : 
 
       HOUSE_INCOME, TRAVEL_TERM, TRAVEL_NUM, TRAVEL_LIKE_SIDO_1,
       TRAVEL_LIKE_SGG_1, TRAVEL_LIKE_SIDO_2, TRAVEL_LIKE_SGG_2,
       TRAVEL_LIKE_SIDO_3, TRAVEL_LIKE_SGG_3, TRAVEL_STYL_1,
       TRAVEL_STYL_2, TRAVEL_STYL_3, TRAVEL_STYL_4, TRAVEL_STYL_5,
       TRAVEL_STYL_6, TRAVEL_STYL_7, TRAVEL_STYL_8,
       TRAVEL_STATUS_RESIDENCE, TRAVEL_STATUS_DESTINATION,
       TRAVEL_STATUS_ACCOMPANY, TRAVEL_STATUS_YMD, TRAVEL_MOTIVE_1,
       TRAVEL_MOTIVE_2, TRAVEL_MOTIVE_3, TRAVEL_COMPANIONS_NUM],

## 전처리

- 필요 column 추출 & null 있는 행 제거

In [ ]:
df_filter = db_travel[~db_travel['TRAVEL_MISSION_CHECK'].isnull()].copy()

df_filter.loc[:, 'TRAVEL_MISSION_INT'] = df_filter['TRAVEL_MISSION_CHECK'].str.split(';').str[0].astype(int)

df_filter.info()

In [ ]:
travel_log_filtered = df_filter[[
    'GENDER',
    'AGE_GRP',
    
    'TRAVEL_STYL_1', 'TRAVEL_STYL_2', 'TRAVEL_STYL_3', 'TRAVEL_STYL_4', 'TRAVEL_STYL_5', 'TRAVEL_STYL_6', 'TRAVEL_STYL_7', 'TRAVEL_STYL_8',
    'VISIT_AREA_NM','TRAVEL_MISSION_CHECK','TRAVEL_MISSION_INT', 'STARS']]




#travel_log_filtered.loc[:, 'GENDER'] = travel_log_filtered['GENDER'].map({'남': 0, '여': 1})

travel_log_filtered = travel_log_filtered.dropna()

travel_log_filtered.head()

In [ ]:
travel_log_filtered.info()

- categorical feature 정의

In [ ]:
# categorical_features_names 리스트에 범주형 특성 이름들을 저장
# CatBoost에서 categorical feature들을 알려주기 위한 용도
# CatBoost는 categorical 인 feature명만 알려주면 자체적으로 변경하여 사용
categorical_features_names = [
    'GENDER'
    #'AGE_GRP',
    ,'TRAVEL_STYL_1', 'TRAVEL_STYL_2', 'TRAVEL_STYL_3', 'TRAVEL_STYL_4', 'TRAVEL_STYL_5', 'TRAVEL_STYL_6', 'TRAVEL_STYL_7', 'TRAVEL_STYL_8'
    ,'TRAVEL_MISSION_INT','VISIT_AREA_NM'
]

# travel_log_filtered 데이터프레임에서 categorical_features_names 리스트의 두 번째 요소부터 마지막에서 두 번째 요소까지의 범주형 특성들을 정수형으로 변환
travel_log_filtered[categorical_features_names[1:-1]] = travel_log_filtered[categorical_features_names[1:-1]].astype(int)

travel_log_filtered.info()

In [ ]:
travel_log_filtered['STARS'].value_counts()

- y 값에 불균형이 이 있어 데이터가 상위 점수 쪽으로 과적합 될 수 있음
- 모델의 하이퍼 파라미터 튜닝에서 이를 조절할 수 도 있으나, 데이터 셋 자체가 작은 편이므로 샘플링을 활용하는 것이 더 적합해 보임

### 데이터 불균형 해소(feat. 언더샘플링 & 오버샘플링)

- 데이터의 불균형이 심해 모델이 높은 점수로 예측할 가능성이 높음
- 따라서 데이터 불균형 문제를 해결해야 함
- 현재 데이터의 경우 4.6만건으로 데이터의 볼륨이 크지 않아 언더샘플링을 하는 것은 데이터의 손실이 너무 커져서 부적절함
- 반대로 오버샘플링을 하기엔 1~3 클래스의 데이터가 너무 적어 데이터의 특성을 살리지 못할 수 있음
- 이 경우 언더 샘플링과 오버샘플링을 조합하는 것이 더 적합해 보임
    - 다수 클래스인 4,5의 데이터 일부에는 언더 샘플링을 적용(비율을 낮게 설정)
    - 소수 클래스인 1~3은 오버 샘플링을 적용해 합성 데이터를 생성
        - 단, 클래스 3은 1~2에 비해 상대적으로 데이터의 수가 훨씬 많으므로 비율을 다르게 적용
- 언더샘플링과 오버샘플링을 조합함으로써 데이터 손실을 최소화 하면서 클래스 불균형 문제를 해소 시도

In [ ]:
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import SMOTENC
from sklearn.preprocessing import LabelEncoder

# 독립 변수와 종속 변수 분리
X = travel_log_filtered.drop('STARS', axis=1)
y = travel_log_filtered['STARS']

# 모든 독립 변수를 범주형 변수로 지정
categorical_features = X.columns[:-1].tolist() # TRAVEL_MISSION_INT 제외한 모든 변수 범주형

# 범주형 변수 전처리
label_encoders = {}

for feature in categorical_features:
    le = LabelEncoder()
    X[feature] = le.fit_transform(X[feature].astype(str))
    label_encoders[feature] = le

# 독립 변수와 종속 변수 분리
y = travel_log_filtered['STARS']

# 언더샘플링 적용
under_sampler = RandomUnderSampler(sampling_strategy={5.0: 15000, 4.0: 15000})
X_under, y_under = under_sampler.fit_resample(X, y)

# 오버샘플링 적용
categorical_indices = [X.columns.get_loc(col) for col in categorical_features]
over_sampler = SMOTENC(sampling_strategy={1.0: 5000, 2.0: 5000, 3.0: 10000}, categorical_features=categorical_indices)
X_resampled, y_resampled = over_sampler.fit_resample(X_under, y_under)

print("Resampled STARS Distribution:")
print(y_resampled.value_counts())

## Train / Test Split

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.2, random_state=42)

print(f"X train shape: {X_train.shape}")
print(f"y train shape: {y_train.shape}")
print(f"X_testshape: {X_test.shape}")
print(f"y_test shape: {y_test.shape}")

In [ ]:
X_train.info()

In [ ]:
X_train.head()

## Modeling

### CatBoost 모델 사용(AI Hub 모델과 동일 알고리즘)

### 모델 정의 및 학습

In [ ]:
from catboost import CatBoostRegressor, Pool

# 학습 데이터 Pool 생성
train_pool = Pool(X_train, label=y_train, cat_features=categorical_features)

# 테스트 데이터 Pool 생성
test_pool = Pool(X_test, label=y_test, cat_features=categorical_features)

# 탐색할 하이퍼파라미터 공간 정의
param_dist = {
    'depth': [4, 6, 8],
    'learning_rate': [0.01, 0.05, 0.1],
    'l2_leaf_reg': [1, 3, 5, 7, 9]
}

# Randomized Search 수행
model = CatBoostRegressor(
    loss_function='RMSE',
    eval_metric='MAE',
    task_type='GPU',
    n_estimators=2000,
    random_seed=42
)

model.randomized_search(
    param_distributions=param_dist,
    X=train_pool,
    y=None,
    cv=5,
    n_iter=20,
    plot=True
)

# 최적의 하이퍼파라미터로 모델 학습
model.fit(train_pool, eval_set=test_pool, verbose=500, plot=True)

## 모델 성능 확인

In [ ]:
# 테스트 데이터에 대한 예측
y_pred = model.predict(X_test)

# 평가 지표 계산
from sklearn.metrics import mean_absolute_error, mean_squared_error, accuracy_score

mae = mean_absolute_error(y_test, y_pred)
rmse = mean_squared_error(y_test, y_pred, squared=False)


print(f"MAE: {mae:.4f}")
print(f"RMSE: {rmse:.4f}")

In [ ]:
import numpy as np
from sklearn.metrics import ndcg_score

# 모델 예측 수행
y_pred = model.predict(X_test)

# 예측 결과를 반올림하여 1에서 5 사이의 정수로 변환
y_pred_rounded = np.round(y_pred)
y_pred_rounded = np.clip(y_pred_rounded, 1, 5)

# 실제 상호작용 데이터와 비교하여 평가 지표 계산
# 예측값 기반 상위 10개 아이템 추출
top_k_indices = np.argsort(-y_pred_rounded)[:10]

# 실제 값에서도 상위 10개 추출
top_k_actual = np.argsort(-y_test)[:10]

# 상위 10개 예측 점수를 2차원 배열로 변환
predicted_scores_top_k = y_pred_rounded[top_k_indices].reshape(1, -1)

# 실제 관련성 점수 배열도 2차원 배열로 변환
actual_relevance = np.zeros_like(y_test)
actual_relevance[top_k_actual] = 1
actual_relevance = actual_relevance[top_k_indices].reshape(1, -1)

# NDCG@10 계산
ndcg_at_10 = ndcg_score(actual_relevance, predicted_scores_top_k, k=10)

# Recall@10 계산
recall_at_10 = len(np.intersect1d(top_k_indices, top_k_actual)) / 10.0

print(f"Recall@10: {recall_at_10:.4f}")
print(f"NDCG@10: {ndcg_at_10:.4f}")

- **MAE** :
    - MAE가 0에 가까울수록 예측의 정확도가 높다는 것을 의미합니다. 여기서 MAE가 0.6210이라는 것은 모델 예측이 평균적으로 실제 값과 0.6210만큼의 차이를 보인다는 것으로 모델의 성능이 상당히 좋다는 것을 알 수 있습니다.
- **RMSE** :
    RMSE 는 예측의 오차크기를 나타내며, 일반적으로 RMSE가 낮을 수록 예측 성능이 높다고 평가합니다. 현재 모델의 경우  평균적으로 실제 값과의 차이가 약 0.7686 정도라는 것을 의미하며, 이 역시 상대적으로 좋은 성능이라 할 수 있습니다.
    
- 결론적으로 현재 구축된 모델의 성능은 상당히 뛰어나다고 할 수 있습니다. 오차가 거의 없이 예측을 하고 있기 때문입니다.

### 테스트

In [ ]:
X_test.reset_index(drop=True, inplace=True)
y_test.reset_index(drop=True, inplace=True)

In [ ]:
# 0부터 9999까지의 범위에서 랜덤한 값을 선택합니다.
import random
i = random.randint(0, 9999)

print(X_test.loc[i])

print(f' \n실제 해당 여행객이 평가한 만족도 점수 : {y_test.loc[i]}')
# DataFrame 형태를 유지하기 위해 loc[[i]]를 사용합니다.
print(f' \n모델이 평가한 여행객의 만족 예상 점수 : {model.predict(X_test.loc[[i]])}')

In [ ]:
def model_test():
    i = random.randint(0, 9999)

    print(X_test.loc[i])

    print(f' \n실제 해당 여행객이 평가한 만족도 점수 : {y_test.loc[i]}')
    # DataFrame 형태를 유지하기 위해 loc[[i]]를 사용합니다.
    print(f' \n모델이 평가한 여행객의 만족 예상 점수 : {model.predict(X_test.loc[[i]])}')

In [ ]:
model_test()

In [ ]:
for i in range(10):
    model_test()

### Feature Importance

In [ ]:
model.get_feature_importance(prettified=True)

## 모델 저장

In [ ]:
# 모델 저장
model.save_model('./model/CatBoost.cb')

## 원본데이터에 대한 모델 성능 테스트

- 위에서 실험한 Train / Test 데이터는 모두 샘플링이된 데이터이므로 테스트의 신뢰도가 낮을 수 밖에 없음
- 샘플링하지 않은 원본데이터를 랜덤하게 가져와 모델의 성능을 테스트

In [ ]:
# 저장된 모델 불러오기
from catboost import CatBoostRegressor
loaded_model = CatBoostRegressor()  
loaded_model.load_model('../model/CatBoost.cb')

In [ ]:
import random

actual_scores = []
predicted_scores = []

def model_test(df):

    while True:
        try:
            i = random.randint(0, len(df) - 1)

            print(df.loc[i])

            actual_scores.append(df.iloc[i, -1])

            # 예측 값 계산 및 출력
            predicted_score = loaded_model.predict(df.iloc[i, :-1].to_numpy().reshape(1, -1))
            predicted_scores.append(predicted_score)

            print(f"\n실제 해당 여행객이 평가한 만족도 점수: {df.iloc[i, -1]}")
            print(f"\n모델이 평가한 여행객의 만족 예상 점수: {predicted_score}\n")
            print('-'*100)

            break

        except KeyError:
            pass
        
        return actual_scores, predicted_scores    

In [ ]:
travel_log_filtered['AGE_GRP'] = travel_log_filtered['AGE_GRP'].astype(str)

In [ ]:
# 예시 실행
model_test(travel_log_filtered.copy())

In [ ]:
# 예시 실행
for i in range(10):
    model_test(travel_log_filtered.copy())

# MAE, RMSE, MSE 계산 및 출력
actual_scores = np.array(actual_scores)  # 리스트를 NumPy 배열로 변환
predicted_scores = np.array(predicted_scores)  # 리스트를 NumPy 배열로 변환

mae = np.mean(np.abs(actual_scores - predicted_scores))
rmse = np.sqrt(np.mean((actual_scores - predicted_scores) ** 2))
mse = np.mean((actual_scores - predicted_scores) ** 2)

print(f"\nMAE: {mae}")
print(f"\nRMSE: {rmse}")
print(f"\nMSE: {mse}")

- 기존 모델 점수
MAE: 0.6210
RMSE: 0.7686